# Cygnus X-3 analysis notebook

## Prerequisities: in order to run this notebook you need the AGILE data.

Hypothesis: there is a probable flare from Cygnus X-3. Check http://www.astronomerstelegram.org/?read=13458

In [ ]:
from agilepy.api.AGAnalysis import AGAnalysis

In [ ]:
%matplotlib notebook

## Analysis name

In [ ]:
user = "bulgarelli"
sourceName = "CYGX3"
analysisname = user+"_"+sourceName

In [ ]:
print("Analysis name:",analysisname)

## Configuration

In [ ]:
tmin = 58884.0 # 2020-02-05 00:00:00.000
tmax = 58886.0 # 2020-02-07 00:00:00.000
timetype = "MJD"
glon = 79.8
glat = 0.7
outDir = f"$HOME/agilepy_analysis/{analysisname}"

In [ ]:
config = """
input:
  evtfile: /AGILE_PROC3/FM3.119_ASDC2/INDEX/EVT.index
  logfile: /AGILE_PROC3/DATA_ASDC2/INDEX/LOG.log.index

output:
  outdir: %s
  filenameprefix: cygnus_atel_analysis
  logfilenameprefix: cygnus_atel_analysis_log
  verboselvl: 1

model:
  galcoeff: -1
  isocoeff: -1

maps:
  binsize: 0.25

selection:
  tmin: %f
  tmax: %f
  timetype: %s
  glon: %f
  glat: %f

mle:
  loccl: 95
"""%(outDir, tmin, tmax, timetype, glon, glat)

with open("./agilepy_conf.yaml", "w") as cf:
    cf.write(config)  

## Obtaining the AGAnalysis object

In [ ]:
ag = AGAnalysis("./agilepy_conf.yaml")

## Loading sources from file

In [ ]:
sources_hypotesis = """
15.7017e-08 80.3286 1.12047 2.16619 0 2 _2AGLJ2032+4135 0 0 0 0 0.5 5 20 10000 0 100
16.9737e-08 79.3 0.55 2.1 3 2 CYGX3 0 0 0 0 0.5 5 20 10000 0 100
119.303e-08 78.2375 2.12298 1.75823 0 2 _2AGLJ2021+4029 0 1 3307.63 0 0.5 5 20 10000 0 100
"""

sourceFile = "./sources.txt"

with open(sourceFile, "w") as sf:
    sf.write(sources_hypotesis)

In [ ]:
sources = ag.loadSourcesFromFile(sourceFile)

In [ ]:
for s in sources:
    print(s)

## Estimation of isotropic and galactic background coefficients

In [ ]:
galBkg, isoBkg, maplistfile = ag.calcBkg('CYGX3', galcoeff = None, pastTimeWindow = 14)

In [ ]:
!cat $maplistfile

## Map generation

In [ ]:
ag.printOptions("maps")

In [ ]:
maplistfile = ag.generateMaps()

In [ ]:
!cat $maplistfile

## Displaying the sky maps

In [ ]:
ag.displayCtsSkyMaps(smooth=3)

In [ ]:
ag.displayExpSkyMaps()

In [ ]:
ag.displayGasSkyMaps()

## Maximum likelihood analysis

In [ ]:
cygx3 = ag.selectSources('name == "CYGX3"').pop()

In [ ]:
ag.mle()

In [ ]:
sources = ag.selectSources("sqrtTS > 0")

In [ ]:
if ag.updateSourcePosition("CYGX3"):
    print("Source position updated")
    print(cygx3)

In [ ]:
ag.mle()

In [ ]:
sources = ag.selectSources("sqrtTS > 0")

In [ ]:
sources = ag.freeSources('name == "CYGX3"', "pos", False)

In [ ]:
ag.mle()

In [ ]:
sources = ag.selectSources("sqrtTS > 0")